In [1]:
import pandas as pd
import numpy as np

In [3]:
austin = pd.read_csv("Data/Austin_Animal_Center_Outcomes.csv")
bloomington = pd.read_csv("Data/Bloomington_Animal_Shelter_Animals.csv")
louisville = pd.read_csv("Data/Louisville_Animal_IO_Data_5.csv")

In [2]:
dallas14 = pd.read_csv("Data/Dallas_Animal_Shelter_Data_Fiscal_Year_2014.csv", dtype={'Tag Type': 'string', 'Activity Number': 'string', "Service Request Number":'string'})
dallas15 = pd.read_csv("Data/Dallas_Animal_Shelter_Data_Fiscal_Year_2015.csv", dtype={'Tag Type': 'string', 'Activity Number': 'string', "Service Request Number":'string'})
dallas16 = pd.read_csv("Data/Dallas_Animal_Shelter_Data_Fiscal_Year_2016.csv", dtype={'Tag Type': 'string', 'Activity Number': 'string', "Service Request Number":'string'})
dallas17 = pd.read_csv("Data/Dallas_Animal_Shelter_Data_Fiscal_Year_2017.csv", dtype={'Tag Type': 'string', 'Activity Number': 'string', "Service Request Number":'string'})
dallas18 = pd.read_csv("Data/Dallas_Animal_Shelter_Data_Fiscal_Year_2018.csv", dtype={'Tag Type': 'string', 'Activity Number': 'string', "Service Request Number":'string'})
dallas19 = pd.read_csv("Data/Dallas_Animal_Shelter_Data_Fiscal_Year_2019.csv", dtype={'Tag Type': 'string', 'Activity Number': 'string', "Service Request Number":'string'})
dallas20 = pd.read_csv("Data/Dallas_Animal_Shelter_Data_Fiscal_Year_2020.csv", dtype={'Tag Type': 'string', 'Activity Number': 'string', "Service Request Number":'string'})

## Ongoing Data Wrangling To Do List

- remove animals that aren't cats and dogs
- remove all rows without outcomes
- standardize column names across locations
- add location columns for each dataframe
- make sure animal IDs don't overlap with others, might need to merge it with their location for unique IDs
- format Dates, Times, length of stay, age across each dataset
- merge Dallas together into one dataframe
- plot years of data to determine if there's enough data from the first recorded years in each location
- standardize categorical intake and outcomes entries across each location